### Import the necessary libraries

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

### Read data

In [33]:
df = gpd.read_file('final_combined_divisions.shp')

### Feature engineering

In [34]:
#convert District to factor
df['District'] = df['District'].astype('category')
df['District'] = df['District'].cat.codes

In [36]:
df.head()

,District,x,y,Year,Area_sq_km,HDI,Cases,Population,Pop_den,geometry
0,0,107.610841,-7.099969,2020,1767.96,72.39,9180,14495160,8198.805403,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427..."
1,0,107.610841,-7.099969,2021,1767.96,72.73,8008,14662620,8293.524740,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427..."
2,0,107.610841,-7.099969,2022,1767.96,73.16,16764,14830092,8388.250865,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427..."
3,0,107.610841,-7.099969,2023,1767.96,73.74,4020,14997564,8482.976990,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427..."
4,1,107.414953,-6.897056,2020,1305.77,68.08,3864,7153344,5478.257273,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688..."


In [43]:
#fit random forest model
X = df.drop(columns='Cases')  # Features
y = df['Cases']

# Stratify based on both 'District' and 'Year'
X_train, X_test, y_train, y_test, train_index, test_index = train_test_split(
    X, y, df.index, test_size=0.2, random_state=42)

# Now you have train_index and test_index containing the original indices
# of the rows that went into training and testing sets respectively.

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [39]:
#OOB error to find the optimal n_estimators, with 500, 1000 trees; (dont need to tune max_features first)
# Define your random forest regressor
rf = RandomForestRegressor(oob_score=True, random_state=42)

# Set up the grid search parameters for n_estimators
param_grid = {
    'n_estimators': [100, 200, 500, 1000],  # Adjust as needed
}

# Set up GridSearchCV with oob_score
grid_search = GridSearchCV(rf, param_grid, cv=None, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit the grid search
grid_search.fit(X_train, y_train)

# Get the best number of trees (n_estimators)
best_n_estimators = grid_search.best_params_['n_estimators']
print(f"Optimal n_estimators: {best_n_estimators}")

# Access the best OOB score
best_oob_score = grid_search.best_estimator_.oob_score_
print(f"Best OOB score: {best_oob_score}")

Optimal n_estimators: 1000
Best OOB score: 0.396834349976112


In [40]:
#compute the predictions and then append the predictions to the df, then extract the df out
rf = RandomForestRegressor(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [47]:
# Add the predictions to the test set (preserving indices)
X_test['Predicted_Cases'] = y_pred

# Now merge predictions back to the original dataframe using the indices of X_test
df_with_predictions = pd.merge(df, X_test[['Predicted_Cases']],
                                left_index=True, right_index=True, how='left')

# Show the dataframe with predictions included
print(df_with_predictions)

#save the df with predictions
df_with_predictions.to_csv("df_combined_with_predictions_rf.csv")

    District           x         y  Year  Area_sq_km    HDI  Cases  \
0          0  107.610841 -7.099969  2020     1767.96  72.39   9180   
1          0  107.610841 -7.099969  2021     1767.96  72.73   8008   
2          0  107.610841 -7.099969  2022     1767.96  73.16  16764   
3          0  107.610841 -7.099969  2023     1767.96  73.74   4020   
4          1  107.414953 -6.897056  2020     1305.77  68.08   3864   
..       ...         ...       ...   ...         ...    ...    ...   
91        22  107.980850 -6.825066  2023     1518.33  73.18   5232   
92        23  108.141296 -7.496706  2020     2551.19  65.67    716   
93        23  108.141296 -7.496706  2021     2551.19  65.90    188   
94        23  108.141296 -7.496706  2022     2551.19  66.84   1276   
95        23  108.141296 -7.496706  2023     2551.19  67.76    352   

    Population      Pop_den  \
0     14495160  8198.805403   
1     14662620  8293.524740   
2     14830092  8388.250865   
3     14997564  8482.976990   
4   